In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

SOURCE_DIR = 'gdrive/MyDrive/ACL/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import torch
import re

In [6]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [5]:
def find_k_nearest_neighbors(word, embedding_dict, k):
  words_cosine_similarity = dict()
  for token in embedding_dict.keys():
    words_cosine_similarity[token] = cos(embedding_dict[word], embedding_dict[token]).item()
  words_cosine_similarity = dict(sorted(words_cosine_similarity.items(), key=lambda item: item[1]))
  return list(words_cosine_similarity.keys())[-k:][::-1]

def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

In [7]:
word = 'زندگی'
k = 10

#0. Data preprocessing

In [8]:
!pip install json-lines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import json_lines

In [9]:
# 1. extract all tweets from files and save them in memory base on each year.
import pandas as pd
df = pd.read_csv('mahsa_amini_data.csv', usecols= ['Text'])
#print(df['Text'][0])
# 2. remove urls, hashtags and usernames.
data = [delete_hashtag_usernames(delete_url(text)) for text in df['Text']]
print(data)

['بنشین تا شود نقش فال ما نقش هم\u200c فردا شدن', 'این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده از بس پای منبر دستمال کشی کرده.', 'برای ایران، برای مهسا.', 'مرگ بر دیکتاتور', 'نذاریم خونشون پایمال شه.\u200c\u200c.\u200c\u200c.', 'مابهت افتخار میکنیم نبات باعث شدی کل دنیا مارو ببینه', 'برای انسانای خوشگلمون', 'فارغ از هر باوری متحد شویم.', 'اینها عجب موجودات پستی هستن🥺🥺🥺الهی بگردم، من خودم باردارم و حتی توتظاهرات مسالمت امیز خارج ایران استرس داشتم ادم ها نا خود اگاه بهم ضربه بزنن،بمیرم برای دل اون زن که چه کشیده...مرگ بر دیکتاتور', 'کصخلا چرا ۴ تاوفحشش نمیدن؟', 'و از روی پول ملی....', 'برای آزادی ایران تا آخرین قطره خونم میجنگم 🇮🇷✌🏻 (7)', 'داریم چهل تا میشیم', 'سی و سه', 'سلام بچه ها اگر من رو میبینید حتما هشتگ بزنید.', 'امروز برای کاری رفتم سراغ وسایل قدیمی و اینارو دیدم : برای تمام روزهایی که رفتم دکه تا کارت دیال آپ بخرم و یه چیزایی تو وبلاگم بنویسم و 88 زدید آرشیوم و حذف کردید برای اون روزها خیلی بغض دارم.', 'فالو کنید بک بدم', 'زن،زندگی،آزادی.', 'دستاشو مشت کرده✊', 'بچه ها را تا 

#1. One hot encoding

In [5]:
# 1. find one hot encoding of each word for each year
import numpy as np
distinct_words = list(set([word for snt in data for word in snt.split(' ')]))
n = len(distinct_words)
one_hot = dict()  
for i, word in enumerate(distinct_words):
  array = np.zeros(n)  
  array[i] = 1
  one_hot.update({word : torch.tensor(array)})
# 2. find 10 nearest words from "ولنتاین"
print(find_k_nearest_neighbors('زندگی', one_hot, 10))

['زندگی', 'ا:ایران', 'پولش', 'بودبراسردشدن', 'دوشانِ', 'خوبیه:)', 'حالم', 'بدبختای', 'جداگانه', 'رنگ.']


#2. TF-IDF

In [11]:
from os import ST_NOATIME
import math
import numpy as np

# 1. find the TF-IDF of all tweets.
tf_all = list()
#data=['he is good', 'he is he']
idf = dict()
for snt in data:
  for word in snt.split(' '):
    idf.update({word: 0})
for snt in data:
  tf = dict()
  for word in snt.split(' '):
    tf.update({word: 0})
  for word in snt.split(' '):
    tf.update({word: int(tf[word]) + 1})
    idf.update({word: int(idf[word] + 1)})
  n = len(snt.split(' '))
  for word in tf:
    tf.update({word: float(tf[word]) / n})
  tf_all.append(tf)
unique = len(idf)
list_word = dict()
for i, word in enumerate(idf):
  idf.update({word: math.log(unique/idf[word])})
  list_word.update({word: i})

array = np.zeros((len(data), unique))
for i,snt in enumerate(data):
  for word in snt.split(' '):
    array[i, list_word[word]] = tf_all[i][word] * idf[word]

dict_tf_idf = dict()
for i, snt in enumerate(data):
  dict_tf_idf.update({snt: torch.tensor(array[i])})

# 2. choose one tweets randomly.
# 3. find 10 nearest tweets from chosen tweet.
find_k_nearest_neighbors(data[1], dict_tf_idf, 10)

['این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده از بس پای منبر دستمال کشی کرده.',
 'برای پدرم که به خاطر کرونا داشت دچار زوال عقل میشد و شده بود سی کیلو اما طاقت آورد...',
 'دچار آزادی شده ایم!!! دچار یعنی عاشق !!!!',
 'برای دستای بی دستمال',
 'سرکارخانم پرستو گوزو در حاله ماله کشی ریدم دهن آدم دروغگو',
 'دیگه هرچی بالای منبر بگین کیرمونم نیست🖕',
 'زنی خفته در خاک ایران را بیدار کرده.',
 'از بس ذهن کثافتی دارن',
 'به نیت رهایی منبر رسول الله و روضه سیدالشهدا',
 'ازادی از رگ گردن نزدیک تر است']

#3. Word2Vec

In [ ]:
# 1. train a word2vec model base on all tweets for each year.
# 2. find 10 nearest words from "ولنتاین"

#4. Contextualized embedding

In [ ]:
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"

In [ ]:
# 1. fine tune a bert model base on all tweets for each year.
# 2. find 10 nearest words from "ولنتاین"